In [1]:
import requests
from pydantic import BaseModel, Field
from typing import Optional

In [2]:
# !pip install pydantic

In [3]:
class WeatherParams(BaseModel):
    city: str = Field(..., description="The name of the city to get weather information for.")
    forecast_type: str = Field("current", description="Either 'current' for current weather or 'forecast' for future weather.")

def get_weather(params: WeatherParams):
    api_key = 'c6dfc4d92a8f972d237ef696ec87b37a'
    base_url = "http://api.openweathermap.org/data/2.5"
    
    if params.forecast_type == "current":
        endpoint = f"{base_url}/weather"
    else:
        endpoint = f"{base_url}/forecast"
    
    params_dict = {
        "q": params.city,
        "appid": api_key,
        "units": "metric"
    }
    
    response = requests.get(endpoint, params=params_dict)
    
    if response.status_code == 200:
        data = response.json()
        if params.forecast_type == "current":
            temp = data['main']['temp']
            description = data['weather'][0]['description']
            return f"The current weather in {params.city} is {description} with a temperature of {temp}°C."
        else:
            forecast = data['list'][0]  # Get the first forecast entry
            temp = forecast['main']['temp']
            description = forecast['weather'][0]['description']
            date_time = forecast['dt_txt']
            return f"The forecast for {params.city} on {date_time} is {description} with a temperature of {temp}°C."
    else:
        return f"Error: Unable to fetch weather data for {params.city}. Status code: {response.status_code}"

In [7]:
from llama_cpp_agent import LlamaCppFunctionTool, FunctionCallingAgent, MessagesFormatterType
from llama_cpp import Llama

# Initialize the Llama model
model_path = "../models/phi-3-gguf/Modelfile_q4"  # Update this to your actual model path
llm = Llama(
    model_path=model_path,
    n_ctx=4096,
    n_threads=8,
    n_gpu_layers=0,
)

# Define the tool using LlamaCppFunctionTool
weather_function_tool = LlamaCppFunctionTool.from_pydantic(get_weather, WeatherParams)

# Create the function calling agent
function_call_agent = FunctionCallingAgent(
    llm,
    llama_cpp_function_tools=[weather_function_tool],
    send_message_to_user_callback=lambda message: print(message),
    allow_parallel_function_calling=True,
    messages_formatter_type=MessagesFormatterType.CHATML
)

# Define the user input
user_input = "What's the current weather in London?"

# Run the agent
response = function_call_agent.generate_response(user_input)
print(response)


gguf_init_from_file: invalid magic characters 'FROM'
llama_model_load: error loading model: llama_model_loader: failed to load model from ../models/phi-3-gguf/Modelfile_q4

llama_load_model_from_file: failed to load model


ValueError: Failed to load model from file: ../models/phi-3-gguf/Modelfile_q4